In [1]:
import pandas as pd
from pathlib import Path
import re

Merge BBR info

In [2]:
data_dir = Path('C:/Users/JBH/Dropbox/11_semester/Public_Econ_seminar/data/bbr')
full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.pq')
)

full_df.to_parquet('C:/Users/JBH/Dropbox/11_semester/Public_Econ_seminar/data/bbr.pq')

In [3]:
full_df=full_df.drop_duplicates()
full_df.to_parquet('data/bbr.pq')

In [4]:
bbr_df = pd.read_parquet('data/bbr.pq')

In [5]:
boliga_df = pd.read_parquet('C:/Users/JBH/Dropbox/11_semester/Public_Econ_seminar/data/boliga.pq')
boliga_df['address'].str.rstrip()
boliga_df['address'].str.lstrip()

0         Finsensvej 47B, 4. 410 2000 Frederiksberg
2                        Tyttebærvej 4 9560 Hadsund
3                   Smedegårdvej 21E 7500 Holstebro
4                         Syrenvej 18 8400 Ebeltoft
5                         Skiffardvej 9 8560 Kolind
                            ...                    
971293                 Struergårdvej 20 7600 Struer
971294                Søgårdsvej 8 4160 Herlufmagle
971295               Landlystvej 25 9210 Aalborg SØ
971296                    Langelinie 6 6000 Kolding
971297                Grantoften 14 3630 Jægerspris
Name: address, Length: 830760, dtype: object

In [6]:
df_kbh = boliga_df[boliga_df['city'].str.contains('København')]
df_aar = boliga_df[boliga_df['city'].str.contains('Aarhus')]
df_ode = boliga_df[boliga_df['city'].str.contains('Odense')]

In [7]:
df_sub = pd.concat([df_kbh, df_aar, df_ode])
df_sub=df_sub.reset_index(drop=True)
regex = r'^\w+'
df_sub['kommune']=df_sub['city'].apply(lambda x: re.findall(regex, x)[0] if re.findall(regex, x) else '')+' Kommune'
df_sub['kommune']=df_sub['kommune'].replace('København Kommune', 'Københavns Kommune')
df_sub

,link,bbr_link,address,price,date_of_sale,type_of_sale,house_size_m2,house_price_per_m2,no_of_rooms,year_built,year,month,week,day,quarter,time_q,post_no_city,post_no,city,kommune
0,https://www.boliga.dk/salg/info/101/533972/416...,https://www.boliga.dk/bbrinfo/416D6FE5-45A5-4B...,"Strandboulevarden 98, 3. th 2100 København Ø",790000,1992-11-03,Alm. Salg,128,6172,5,1903,1992,11,45,3,4,1992Q4,2100 København Ø,2100,København Ø,Københavns Kommune
1,https://www.boliga.dk/salg/info/101/187276/933...,https://www.boliga.dk/bbrinfo/933F5DC3-E067-45...,"Grækenlandsvej 37, 1. th 2300 København S",319000,1994-02-14,Alm. Salg,53,6019,2,1934,1994,2,7,14,1,1994Q1,2300 København S,2300,København S,Københavns Kommune
2,https://www.boliga.dk/salg/info/101/705960/1BA...,https://www.boliga.dk/bbrinfo/1BAB9A84-670B-4C...,"Weidekampsgade 59, 4. th 2300 København S",2645000,2006-03-21,Alm. Salg,93,28441,2,2006,2006,3,12,21,1,2006Q1,2300 København S,2300,København S,Københavns Kommune
3,https://www.boliga.dk/salg/info/101/716210/B3B...,https://www.boliga.dk/bbrinfo/B3B0A4DA-8B77-4E...,"Klaksvigsgade 9, 3. tv 2300 København S",3300000,2006-03-21,Alm. Salg,110,30000,3,2006,2006,3,12,21,1,2006Q1,2300 København S,2300,København S,Københavns Kommune
4,https://www.boliga.dk/salg/info/101/707610/408...,https://www.boliga.dk/bbrinfo/408813A5-3EAC-46...,"Weidekampsgade 61, 5. tv 2300 København S",2500000,2006-03-21,Alm. Salg,93,26882,2,2006,2006,3,12,21,1,2006Q1,2300 København S,2300,København S,Københavns Kommune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68985,https://www.boliga.dk/salg/info/461/188923/CDC...,https://www.boliga.dk/bbrinfo/CDC44BEA-0AC5-45...,"Jagtvej 23, 2. tv 5000 Odense C",6900000,2006-03-17,Alm. Salg,65,106154,2,1911,2006,3,11,17,1,2006Q1,5000 Odense C,5000,Odense C,Odense Kommune
68986,https://www.boliga.dk/salg/info/461/188923/922...,https://www.boliga.dk/bbrinfo/922032EA-018A-48...,"Jagtvej 23, 1. tv 5000 Odense C",6900000,2006-03-17,Alm. Salg,65,106154,2,1911,2006,3,11,17,1,2006Q1,5000 Odense C,5000,Odense C,Odense Kommune
68987,https://www.boliga.dk/salg/info/461/608442/ACE...,https://www.boliga.dk/bbrinfo/ACED4E9E-F0FC-44...,Væbnerhatten 54 5220 Odense SØ,2350000,2006-03-18,Alm. Salg,161,14596,6,1985,2006,3,11,18,1,2006Q1,5220 Odense SØ,5220,Odense SØ,Odense Kommune
68988,https://www.boliga.dk/salg/info/461/329977/4AB...,https://www.boliga.dk/bbrinfo/4ABE4DDB-C97A-46...,Rulkehøjen 36 5260 Odense S,2170000,2006-03-19,Alm. Salg,164,13232,6,1970,2006,3,11,19,1,2006Q1,5260 Odense S,5260,Odense S,Odense Kommune


In [8]:
land_tax = pd.read_excel('data/kmn_grundskyldspromille.xlsx')
land_tax['year']=pd.to_datetime(land_tax['year'], format='%Y').dt.year
land_tax

,kommune,year,land_tax
0,Københavns Kommune,1993,34.00
1,Odense Kommune,1993,12.90
2,Aarhus Kommune,1993,18.90
3,Københavns Kommune,1994,34.00
4,Odense Kommune,1994,12.90
...,...,...,...
85,Odense Kommune,2021,21.71
86,Aarhus Kommune,2021,24.58
87,Københavns Kommune,2022,34.00
88,Odense Kommune,2022,21.71


Merge in land tax values

In [9]:
df_sub=pd.merge(df_sub, land_tax, left_on=['kommune','year'], right_on=['kommune','year'])